In [15]:
import numpy as np
import pandas as pd

from collections import Counter
from matplotlib import pyplot as plt

In [8]:
# We should train on a subset of the corpus, with 100 0000 tokens. Find how many lines this corresponds to

filename = "books.txt" 
encoding = "ISO-8859-1"

line_counter = 0
token_counter = 0

with open(filename, encoding = encoding) as f:
    for line in f:

        line_counter += 1
        tokens = line.lower().split()

        for token in tokens:
            token_counter += 1
            
        if token_counter > 10**5:
            break

print(line_counter)

582


In [17]:
def count_word_frequencies(filename, encoding, n_documents):
    
    freqs = Counter()
    with open(filename, encoding = encoding) as f:
        for i, line in enumerate(f):
            
            if i == n_documents:
                break
            
            tokens = line.lower().split()
            for token in tokens:
                freqs[token] += 1
                
    return freqs, i

# 1: Write your own code for doing Gibbs sampling for LDA

In [53]:
beta = 0.1
alpha = 0.1
n_topics = 10

# beta = 0.01
# alpha = 0.01
# n_topics = 50

word_frequencies, n_documents = count_word_frequencies(filename, encoding, line_counter)

# try ignoring the 20 most common words
n_ignore = 20
tmp = word_frequencies.most_common()[0:n_ignore]
ignore_list = list(map(list, zip(*tmp)))[0]

vocab_size = len(word_frequencies) - n_ignore

# assign a random topic from 1:n_topics, for each word not on the ignore_list
# maybe save the key words for each line as well, to make it easier to step through

# I am the night -> night


Assess model performance by perplexity and in tables over the ten top words for the topics.

# 2: Write your own code for doing Gibbs samling on Bigram LDA

as in H. M. Wallach: Topic modeling: beyond bag-of-words. ICML(2006) 977-984. http://dirichlet.net/pdf/wallach06topic.pdf

For corpus use the Amazon book reviews corpus that you also used in Assignment 1. You may have to use only a subset of the documents. A corpus of 100 000 tokens is sufficients size.

Run this for different hyperparameters. For LDA you can try α=β=0.1 and α=β=0.01. (A cross-validation search for optimal values will probably be too slow.) Run also for different numbers of topics, e.g. K=10 and K=50.

For the bigram model, see to that you use a larger hyperparameter value on the diagonal of the transition matrix over the topics. Since each document in this model has a transition matrix over topics rather than just a probability distribution, the number of topics cannot be as large as for LDA. Try K=5 and K=10.